In [ ]:
from PIL import Image
import numpy as np
import scipy

In [ ]:
img1 = Image.open("pic_left.jpg")
img2 = Image.open("pic_right.jpg")

In [70]:
### 在兩張圖片上手動找出三個相同物品的pixel位置
pList1 = [(2779,2747),(1139,2173),(1772,1389)]
pList2 = [(1651,2357),(55,1758),(692,965)]

In [40]:
def createTransform(pList1,pList2):
    ### 產生transformation function
    primary = np.float32(pList1)
    secondary = np.float32(pList2)
    
    n = primary.shape[0] # dim
    pad = lambda x: np.hstack([x, np.ones((x.shape[0], 1))]) #增加維度成 NxN Matrix
    unpad = lambda x: x[:,:-1] # 把維度降回
    X = pad(primary)
    Y = pad(secondary)
    
    A = np.linalg.solve(X,Y)

    transform = lambda x: unpad(np.dot(pad(x), A))
    return transform
    
    
    

In [72]:
transform = createTransform(pList1,pList2)
invers = createTransform(pList2,pList1)

In [ ]:
### 測試用
transform(np.float32([[2779,2747],[1651,2357]]))

In [ ]:
### 測試用
invers(np.float32([[2008,2285],[967,2439]]))

In [13]:
def bilinear(targetTuple,rgb_im):
    x = targetTuple[0]
    y = targetTuple[1]
    
    if (x < 0 or x >= rgb_im.width or y < 0 or y >= rgb_im.height):
        return (0,0,0)
    if(x==rgb_im.width-1 or y==rgb_im.height-1):
        return rgb_im.getpixel((x,y))
    
    x1 = int(x)
    x2 = min(x1+1,rgb_im.width-1)
    y1 = int(y)
    y2 = min(y1+1,rgb_im.height-1)
    
    x11 = np.float32([(x-x1)*p for p in rgb_im.getpixel((x1, y2))]) + np.float32([(x2-x)*p for p in rgb_im.getpixel((x1, y1))])
    #print(x11)
    x22 =np.float32( [(x-x1)*p for p in rgb_im.getpixel((x2, y2))]) + np.float32([(x2-x)*p for p in rgb_im.getpixel((x1, y2))])
    #print(x22)
    y12 = (y-y1) * x22 + (y2-y) * x11
    
    return y12
    

In [14]:
bilinear((4127,3095),img2.convert('RGB'))

(11, 13, 12)

In [74]:
inverPoint = invers(np.float32([[0,0],[0,img2.height],[img2.width,0],[img2.width,img2.height]]))
width = 0;
height = 0;
for points in inverPoint:
    width = max(width,points[0])
    height = max(height,points[1])
width = int(max(img1.width,width))
height = int(max(img1.height,height))
print(width,height)

img1_rgb = img1.convert('RGB')
img2_rgb = img2.convert('RGB')

imgMerge = Image.new('RGB',(width,height),'black')
pixels = imgMerge.load();

#transP = transform(np.float32([[99,111]]))
#p = bilinear(transP[0],img2_rgb)
#print(tuple(p))
#pixels[0,0] = p

for j in range(height):
    for i in range(width):
        if (i < img1.width and j < img1.height):
            pixels[i,j] = img1_rgb.getpixel((i,j))
        else :
            transP = transform(np.float32([[i,j]]))
            p = bilinear(transP[0],img2_rgb)
            pixels[i,j] = tuple(p)
        #print(pixels[i,j])
imgMerge.save('mergeImg2.jpg','JPEG')
imgMerge.show()

4137 3096
